In [1]:
import openai

import os
from pathlib import Path
import random
import time
import json

In [2]:
openai.api_key = 'Enter your OpenAI API key'

In [3]:
def generate(prompt, model='code-davinci-002', **kwargs):

    openai.api_key = openai.api_key or os.getenv("OPENAI_API_KEY")

    while True:
        try:
            res = openai.Completion.create(
                model=model,
                prompt=prompt,
                **kwargs
            )
            return res.choices
        except openai.error.RateLimitError:
            time.sleep(1)
        except openai.error.InvalidRequestError as e:
            return e

In [5]:
prompt = f'''### Buggy Program ###
for t in range(int(input())):
    b, w = tuple(map(int, input().strip().split()))
    if b * w:
        if (b + w) == 1:
            print("W" * w + "B" * b)
        else:
            print(-1)
    else:
        print("B" * b + "W" * w)
        for i in range(1, b + w):
            if b == 1:
                print(1, i+1)
            else:
                print(i, b+w)

### Testcase ###
# input:
1
0 2
# wrong output:
WW
1 2
# correct output:
-1

### Correct Program ###
for t in range(int(input())):
    b, w = tuple(map(int, input().strip().split()))
    if b * w == 0:
        if (b + w) == 1:
            print("W" * w + "B" * b)
        else:
            print(-1)
    else:
        print("B" * b + "W" * w)
        for i in range(1, b + w):
            if b == 1:
                print(1, i+1)
            else:
                print(i, b+w)

### Explain why the first code is wrong, and how to be fixed.
'''

In [ ]:
prompt = f'''### Buggy Program ###
import math, heapq

def k(i):
    return i[1]

for i in range(int(input())):
    n = int(input())
    ai = list(map(int, input().split()))
    ti = list(map(int, input().split()))
    a = [(-ai[i], ti[i], i+1) for i in range(n)]
    a = sorted(a, key=k)

    t = 0
    l = []
    heapq.heapify(l)
    for i in range(n):
        if(a[i][0]+a[i][1] >= 0):
            if(t-a[i][0] <= a[i][1]):
                heapq.heappush(l, (a[i]))
                t -= a[i][0]
            else:
                head = heapq.heappop(l)
                t-=head[0]
                if(head[0] < a[i][0] and t-a[i][0] <= a[i][1]):
                    heapq.heappush(l, (a[i]))
                    t -= a[i][0]
                else:
                    heapq.heappush(l, head)
                    t -= head[0]
    l = sorted(list(l), key=k)
    temp = 0
    print(len(l))
    for i in l:
        print(i[2], temp, temp-i[0])
        temp-=i[0]

### Testcase ###


'''

In [6]:
res = generate(
    prompt,
    max_tokens=1024,
    top_p=0.95,
    n=3,
    # stop=['"""', '# ', '##']
)

In [8]:
res

InvalidRequestError(message='The model: `code-davinci-002` does not exist', param=None, code='model_not_found', http_status=404, request_id=None)

In [7]:
for x in res:
    print(x.text)
    print('--------')

TypeError: 'InvalidRequestError' object is not iterable

In [27]:
for x in res:
    print(x.text)
    print('--------')


You might want to check the logic order first:

- The program mustn't print the (result, indexes) if b*w == 0.
- The program must print the (result, indexes) if b*w > 0.
- The program must print the single string "W" if (b + w) == 1.
- If (b + w) > 1, the result must be "-1".
- Here is the interesting part: You mustn't print nothing if b*w == 0.

So:

- The print() doesn't follow the defined logic. For example, the 'print(result)' must be after the 'if b*w == 0'.

You might want to check the python standard lib. Here's what happends when you call print() with nothing else:

- When the print() is called with no arguments, a newline is printed.
- So it will print a new line to you.

In conclusion: We need to do it this way:

- The case of (b * w > 0), result will print.
- The case of (b * w == 0), the result won't print.
"""

"""
생각 과정:

- b*w == 0 이라면 출력할 것도 없다. 
- b*w > 0 이라면, 출력해야 한다.
- (b + w) == 1 이라면, 출력해야 한다.
- (b + w) > 1 이라면, -1 을 출력해야 한다.
- 재밌는 부분은 이게다. b*w == 0 이라면, 내용을 출력하면 

In [40]:
res

InvalidRequestError(message='Unrecognized request argument supplied: top_k', param=None, code=None, http_status=400, request_id=None)

In [5]:
with open('singles.json', 'r', encoding='utf-8') as f:
    singles = json.load(f)
with open('multis.json', 'r', encoding='utf-8') as f:
    multis = json.load(f)

In [6]:
with open('single_test_sample.txt', 'r') as f:
    single_ids = [x.strip() for x in f.readlines()]
with open('multi_test_sample.txt', 'r') as f:
    multi_ids = [x.strip() for x in f.readlines()]

In [33]:
single_samples = {s: singles[s] for s in single_ids}
multi_samples = {m: multis[m] for m in multi_ids}

In [34]:
single_samples

{'CATSRATS_43': {'id': 'CATSRATS_43',
  'src': '# cook your dish here\nclass Animal:\n    def __init__(self):\n        start, end, starting_time = map(int, input().split())\n        \n        self.ending_time = starting_time + abs(start - end)\n        self.velocity = 1 if end >= start else -1 \n        \n        self.eaten_by = -1, 10 ** 10\n        \n        self.start = start \n        self.end = end \n        self.starting_time = starting_time \n        \n    def will_collide(self, z):\n        if self.starting_time > z.ending_time or self.ending_time < z.starting_time:\n            return False \n            \n        if self.velocity == z.velocity:\n            if self.starting_time > z.starting_time:\n                self, z = z, self\n            if z.start == self.start + self.velocity * (z.starting_time - self.starting_time):\n                return z.starting_time\n            else:\n                return False\n        \n        if self.velocity == -1:\n            self, z

In [9]:
import difflib

In [10]:
def view_sample(sample):
    print(f'[{sample["id"]}]\n--------\n[Wrong]\n{sample["src"]}\n--------\n[Correct]\n{sample["tgt"]}\n--------')
    diff = difflib.context_diff(sample['src'].split('\n'), sample['tgt'].split('\n'))
    print(f'[Diff]\n')
    for l in diff:
        print(l)
    print('====')

In [35]:
single_samples[single_ids[0]]

{'id': 'CATSRATS_43',
 'src': '# cook your dish here\nclass Animal:\n    def __init__(self):\n        start, end, starting_time = map(int, input().split())\n        \n        self.ending_time = starting_time + abs(start - end)\n        self.velocity = 1 if end >= start else -1 \n        \n        self.eaten_by = -1, 10 ** 10\n        \n        self.start = start \n        self.end = end \n        self.starting_time = starting_time \n        \n    def will_collide(self, z):\n        if self.starting_time > z.ending_time or self.ending_time < z.starting_time:\n            return False \n            \n        if self.velocity == z.velocity:\n            if self.starting_time > z.starting_time:\n                self, z = z, self\n            if z.start == self.start + self.velocity * (z.starting_time - self.starting_time):\n                return z.starting_time\n            else:\n                return False\n        \n        if self.velocity == -1:\n            self, z = z, self\n     

In [36]:
view_sample(single_samples[single_ids[1]])

[ANTIKNAPSACK_60]
--------
[Wrong]
t = int(input())
for i in range(t):
    n,k = map(int,input().split())
    res = 0
    for i in range(1,20):
        if not k % i:
            res = i
            break
    for i in range(1,n+1):
        print(res*i , end=" ")
    print()
--------
[Correct]
t = int(input())
for i in range(t):
    n,k = map(int,input().split())
    res = 0
    for i in range(1,20):
        if k % i:
            res = i
            break
    for i in range(1,n+1):
        print(res*i , end=" ")
    print()
--------
[Diff]

*** 

--- 

***************

*** 3,9 ****

      n,k = map(int,input().split())
      res = 0
      for i in range(1,20):
!         if not k % i:
              res = i
              break
      for i in range(1,n+1):
--- 3,9 ----

      n,k = map(int,input().split())
      res = 0
      for i in range(1,20):
!         if k % i:
              res = i
              break
      for i in range(1,n+1):
====


In [65]:
def prompt_src_tgt(desc, src, tgt, tc_in, tc_out):
    prompt = f'''You are the problem finder that should find the algorithmical error between given buggy code and correct code, with the problem description and the test case. The buggy code didn't pass the test case and correct code passed. You have to explain why the buggy code cannot pass that and what is different with the correct code.

The problem description is as below:
{desc}

The buggy code is as below:
```python3
{src}
```

The failed testcase is as below:
- Input:
{tc_in}
- Output:
{tc_out}

The correct code is as below:
```python3
{tgt}
```

The problem in the buggy code is, '''
    return prompt

In [66]:
def prompt_src (desc, code, tc_in, tc_out, line):
    prompt = f'''Question: You are the code analyzer that should find the logical flaw at mentioned line in given code with the testcase. The code didn't pass the testcase, so you have to explain what flaws make the code not to be able to pass that.

The problem description is as below:
{desc}

The code is as below:
```python3
{code}
```
And the code has a flaw in the line:
{line}

The failed testcase is as below:
- Input:
{tc_in}
- Output:
{tc_out}

Answer: The flaw is, '''
    print(prompt)
    return prompt

In [67]:
def get_prompt_src(sample):
    return prompt_src(sample['desc'], sample['src'], sample['tc_in'], sample['tc_out'], sample['line'])

In [68]:
def get_prompt_src_tgt(sample):
    return prompt_src_tgt(sample['desc'], sample['src'], sample['tgt'], sample['tc_in'], sample['tc_out'])

In [69]:
single_samples[single_ids[1]].update(dict(
    desc='''You are given two integers N, K.

Construct an array A consisting of N distinct positive integers such that the following conditions are met:
- 1 <= A_i <= 2 * 10^4
- There exists no subsequence of A with sum K.
- If there are multiple answers, print any.
- 1 <= K <= 20
''',
    tc_in='''2
4 10
10 1000000''',
    tc_out='''1 2 5 6
327 8695 4 19852 14 736 46 15 37 928''',
    line='        if not k % i:'
))

In [70]:

# res = openai.Completion.create(
#     prompt_src(src, tc_in, tc_out),
#     max_tokens=2048,
#     top_p=0.95,
#     n=3,
#     # stop=['"""', '# ', '##']
# )
res = openai.Completion.create(
    model='code-davinci-002',
    prompt=get_prompt_src_tgt(single_samples[single_ids[1]]),
    temperature=0.3,
    max_tokens=1024,
    top_p=0.95,
    n=3
)

In [71]:
for x in res.choices:
    print(x.text)
    print('--------')

the bug is in the for loop.
```python3
for i in range(1,20):
    if not k % i:
        res = i
        break
```

The correct code is as below:
```python3
for i in range(1,20):
    if k % i:
        res = i
        break
```

The reason why the buggy code cannot pass the test case is, the buggy code will find the first factor of k, but the correct code will find the first non factor of k.

The test case is as below:
- Input:
2
4 10
10 1000000
- Output:
1 2 5 6
327 8695 4 19852 14 736 46 15 37 928

The output is as below:
- Output:
1 2 5 6
327 8695 4 19852 14 736 46 15 37 928

The explanation is as below:
- Explanation:
The first test case is simple, the first non factor of 10 is 5.

The second test case is a bit tricky, the first non factor of 1000000 is 327.

The first non factor of 327 is 4.

The first non factor of 4 is 2.

The first non factor of 2 is 1.

The first non factor of 1 is 1.

The first non factor of 1 is 1.

The first non factor of 1 is 1.

The first non factor of 1 is 

In [64]:
view_sample(multi_samples['ENGLISH_900'])

[ENGLISH_900]
--------
[Wrong]
"""
Code Chef :: January 2020 Challenge :: English
https://www.codechef.com/JAN20A/problems/ENGLISH
"""

def stanza_beauty(wd1, wd2):
    """
    Returns the beauty of the stanza.  The beauty of the stanza is 
    the minimum of the lenght of the common prefix and the length
    of the common suffix.
    """
    L = min(len(wd1), len(wd2))
    for i in range(L):
        if wd1[i] != wd2[i] or wd1[-i-1] != wd2[-i-1]:
            return i * i
    return L * L


def transform(wd):
    return "".join("".join((a, b)) for a, b in zip(wd, reversed(wd)))
    
def solve(N, words):
    """Solve puzzle."""
    words = sorted(words, key=transform)
    stanzas = []
    stanzas_append = stanzas.append
    for i in range(N):
        wd1 = words[i]
        j = (i+1) % N
        wd2 = words[j]
        b = stanza_beauty(wd1, wd2)
        if b > 0:
            stanzas_append((b, i, j))

    solution = 0
    taken = set()
    for weight, wd1, wd2 in sorted(stanzas,reverse=Tr

In [2]:
import pickle

with open('codechef_14_assignment_info.pkl', 'rb') as f:
    d = pickle.load(f)

In [3]:
d['abcstr'].keys()

dict_keys(['title', 'tag', 'problem', 'input_tc', 'output_tc', 'problem_token_set', 'public_tests', 'private_tests', 'generated_tests'])